<a href="https://colab.research.google.com/github/BecomeAllan/S2Search/blob/main/SemanticScholarSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consumindo a API do SemanticScholar

A seguir, tem uma classe chamada `Search()`, que ao instanciar-la em uma variável é possível fazer pesquisas sobre papers utilizando a api do SemanticScholar, dentre os parâmetros temos:

- Buscar: Pesquisas sobre tópicos onde adicionar tópicos utiliza-se + (mais) e remover tópicos usamos - (menos)

  ex. "Machine+Medicine"

- Fields: O que será retornado como dados. Para utilizar, escolha dentre as opções sem utilizar espaço e separadas de virgulas:
  - (str): externalIds
  - (str): url
  - (str): title
  - (str): abstract
  - (str): venue 
  - (str): year 
  - (str): referenceCount
  - (str): citationCount
  - (str): influentialCitationCount
  - (str): isOpenAccess
  - list (str): fieldsOfStudy
  - list (str): authors 

  ex. "title,abstract,isOpenAccess,fieldsOfStudy"

- Offset: Número que começa a puxar a partir da ordem dele a lista de papers. (0 seria o primeiro)

- Limite: Número de papers a ser retornados (Máx. 10.000)

**Obs:** A api do SemanticScholar disponibiliza 100 query's a cada 5 min, no qual apenas retorna no máx. 100 resutados (limite). Assim a cada 5 min, é possível puxar 10.000 papers.

In [ ]:
#@title Classe para pesquisa no SemanticScholar
import IPython
from google.colab import output
import pandas as pd

class Search():
  def __init__(self, **kwargs):
    self.data = ""
    self.data_0 = ""

    self.search = kwargs.get('search', None)
    self.fields = kwargs.get('fields', None)
    self.limit = kwargs.get('limit', None)
    self.offset = kwargs.get('offset', None)

    if self.search == None and self.fields == None and self.limit == None and self.offset == None:
      self._start(False)
    else:
      self._start(True)
  
  def _start(self, *args):

    output.register_callback('notebook.searching', self._searching)
    output.register_callback('notebook.AddListItem', self._add_list_item)
    output.register_callback('notebook.mergeData', self._merge_data)
    output.register_callback('notebook.error', self._error)


    boxs = ''' 
        <label for="query">Buscar: </label>
        <input type="text" id="query" value="Machine Learning+Deep Learning" style="width: 400px;"/>
        <br/>
        <br/>
        
        <label for="fields">Fields: </label>
        <input type="text" id="fields" value="title,abstract,isOpenAccess,fieldsOfStudy" style="width: 400px;"/>
        <br/>
        <br/>
 
        <label for="limit">Limite: </label>
        <input type="text" id="limit" value="10" style="width: 50px;"/><br/>
        <br/>

        <label for="limit">Offset: </label>
        <input type="text" id="offset" value="0" style="width: 50px;"/><br/>
        <br/>

        <button id='button'>Pesquisar</button>
        <br/>
        <br/>
           '''

    button = ''' document.querySelector('#button').onclick = async () => ''' # {}

    search_query = '''
            var search = document.getElementById("query").value
            var fields = document.getElementById("fields").value
            var limit = parseInt(document.getElementById("limit").value)
            var offset = parseInt(document.getElementById("offset").value)
                  '''
    search_params = '''
            var search = "{search}"
            var fields = "{fields}"
            var limit = parseInt({limit})
            var offset = parseInt({offset})
                  '''
    engine = '''
            google.colab.kernel.invokeFunction('notebook.searching', [], {});

            if (limit >100) {
              var number = limit
              var data = ""
              var promises = []
              var offsetSearch = 0
              var rest = 0

              for (let index = 0; index < Math.floor(limit/100); index++) {
                offsetSearch = 100*(index) + offset + 1*(index!==0)


                promises.push(
                  fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offsetSearch}&limit=100&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
                )
              }
              
              if (limit%100 !== 0) { 
                rest= limit%100
                offsetSearch = offsetSearch+100
                
                console.log(rest)
                console.log(offsetSearch)

                promises.push(
                fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offsetSearch}&limit=${rest}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
                )}

              await Promise.all(promises).then(data=>{
                google.colab.kernel.invokeFunction('notebook.mergeData', [data], {})
              })
              .catch(err=> { return (google.colab.kernel.invokeFunction('notebook.error', [err], {})) })

            } else {

            await fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offset}&limit=${limit}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {
      console.log(res)
      console.log("AQUIII")
      return(google.colab.kernel.invokeFunction('notebook.AddListItem', [res], {}))})
    .catch(err=> { return (
      google.colab.kernel.invokeFunction('notebook.error', [err], {})) })
            }
                  '''

    asyncfun = "async function asyncfun()"

    if args[0]:

      main_app =  "<script>" + search_params.format(search=self.search, fields=self.fields, limit=self.limit, offset=self.offset) + asyncfun + "{" + engine + "}" + "asyncfun()" + "</script>"

      display(IPython.display.HTML(main_app))
      
    else:
      main_app = boxs + "<script>" + button + "{" + search_query + engine + "}" + "</script>"
      
      display(IPython.display.HTML(main_app))

    

  def _error(self,value):
    try:
      print("ERRO na API SemanticScholar:\n")
      print(value)
    except:
      pass 

  def _searching(self):
    with output.use_tags('some_outputs'):
      print("\n\nPesquisando...")
      sys.stdout.flush();

  def _merge_data(self, data):
    output.clear(output_tags='some_outputs')
    print(f"Achou {data[0]['total']} papers.\n")
    self.data_0 = data

    self.data = pd.DataFrame(data[0]['data'])

    try:
      for x in data[1:len(data)]:
        try:
          self.merge(pd.DataFrame(x['data']))
        except:
          self._error(x)
    except:
      pass 

    print(f"\nApi devolveu >> {self.data.shape[0]} papers\n" )
    print(self.data.head())


  def merge(self, data):
    self.data = pd.concat([self.data, data], ignore_index=True ) 

  def _add_list_item(self, value):
    output.clear(output_tags='some_outputs')

    print(f"Achou {value['total']} papers.\n")

    self.data = pd.DataFrame(value['data'])

    print(f"Api devolveu >> {self.data.shape[0]} papers\n" )
    
    print(self.data.head())



# Consumir a classe `Search()`

A duas formas de pesquisar utilizando `Search()`:

1. A primeira é utilizando parâmetros na propria classe:

In [ ]:
# "Decision making" AND "optimization" AND "artificial intelligence"

In [ ]:
Resultados = Search(search = "Machine Learning+Deep Learning" , fields = "title,abstract,citationCount,isOpenAccess,fieldsOfStudy", limit = "2000", offset = "0")

Achou 5240039 papers.

ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}

Api devolveu >> 1009 papers

                                    paperId  ...                    fieldsOfStudy
0  46200b99c40e8586c8a0f588488ab6414119fb28  ...               [Computer Science]
1  9c9d7247f8c51ec5a02b0d911d1d7b9e8160495d  ...               [Computer Science]
2  b42b1bfdc262bf99e9484e2e9df94df216b96374  ...               [Computer Science]
3  25badc676197a

In [ ]:
# Os dados ficam na variável data, no qual é uma tabela do tipo pandas
print(Resultados.data.columns)
print(Resultados.data.sort_values("citationCount", ascending = False ).head())

,paperId,title,abstract,citationCount,isOpenAccess,fieldsOfStudy
754,34f25a8704614163c4095b3ee2fc969b60de4698,Dropout: a simple way to prevent neural networ...,Deep neural nets with a large number of parame...,24510,False,[Computer Science]
17,a4cec122a08216fe8a3bc19b22e78fbaea096256,Deep Learning,Machine-learning technology powers many aspect...,19531,False,"[Medicine, Computer Science]"
963,5d90f06bb70a0a3dced62413346235c02b1aa086,Learning Multiple Layers of Features from Tiny...,Groups at MIT and NYU have collected a dataset...,13232,False,[Computer Science]
988,6bdb186ec4726e00a8051119636d4df3b94043b5,Caffe: Convolutional Architecture for Fast Fea...,Caffe provides multimedia scientists and pract...,13101,False,[Computer Science]
14,46200b99c40e8586c8a0f588488ab6414119fb28,TensorFlow: A system for large-scale machine l...,TensorFlow is a machine learning system that o...,10598,False,[Computer Science]


2. A segunda é atravez da api de busca, searchBox, no qual é possivel colocar os campos:

In [ ]:
Resultados_2 = Search()

Achou 652200 papers.

Api devolveu >> 10 papers

                                    paperId  ...       fieldsOfStudy
0  846ff7afb7670d62f88b4a8cc99d306ffb81b075  ...          [Medicine]
1  5dc53e50148b01fe8b9536eb79fa6b1dce924174  ...          [Medicine]
2  7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f  ...  [Computer Science]
3  59444b096f7c8a561d540102e8b5bfb189edabc6  ...                None
4  eee313380ccb45807ea0afa3c1df86f6b48b8867  ...  [Computer Science]

[5 rows x 5 columns]


In [ ]:
print(Resultados_2.data)

                                    paperId  ...       fieldsOfStudy
0  846ff7afb7670d62f88b4a8cc99d306ffb81b075  ...          [Medicine]
1  5dc53e50148b01fe8b9536eb79fa6b1dce924174  ...          [Medicine]
2  7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f  ...  [Computer Science]
3  59444b096f7c8a561d540102e8b5bfb189edabc6  ...                None
4  eee313380ccb45807ea0afa3c1df86f6b48b8867  ...  [Computer Science]
5  46479bbea7749cb2db35b139206039531327053c  ...  [Computer Science]
6  b69fe5a837277ddbea5215d6bacd3a902e9d11ce  ...          [Medicine]
7  b0bf64ccbd651e8c7bc141d8aabaecff562e93a1  ...  [Computer Science]
8  042ab08ec6782cf217f13175162bfd48f7350114  ...  [Computer Science]
9  03e7832982986159400a8eeab148487ffcfabe56  ...  [Computer Science]

[10 rows x 5 columns]


# **ON BUILD...**

In [ ]:
#### NOT RUN

## Para fazer a requisição de dados, é utilizado o metodo POST para HTTP
import requests
import json

url = "https://www.semanticscholar.org/api/1/search"

PARAMS_FOR_POST = '''{
    "queryString": "Machine Learning+Deep Learning",
    "page": 1,
    "pageSize": 10,
    "sort": "total-citations", #influence #"pub-date" #relevance
    "authors": [],
    "coAuthors": [],
    "venues": [
        "PloS one",
        "AAAI",
        "Scientific reports",
        "IEEE Access",
        "ArXiv",
        "Expert Syst. Appl.",
        "ICML",
        "Neurocomputing",
        "Sensors",
        "Remote. Sens."
    ],
    "yearFilter": {
        "min": 2008,
        "max": 2021
    },
    "requireViewablePdf": true,
    "publicationTypes": [
        "ClinicalTrial",
        "CaseReport",
        "Editorial",
        "Study",
        "Book",
        "News",
        "Review",
        "Conference",
        "LettersAndComments",
        "JournalArticle"
    ],
    "externalContentTypes": [],
    "fieldsOfStudy": [
        "biology",
        "art",
        "business",
        "computer-science",
        "chemistry",
        "economics",
        "engineering",
        "environmental-science",
        "geography",
        "geology",
        "history",
        "materials-science",
        "mathematics",
        "medicine",
        "philosophy",
        "physics",
        "political-science",
        "psychology",
        "sociology"
    ],
    "useFallbackRankerService": false,
    "useFallbackSearchCluster": false,
    "hydrateWithDdb": true,
    "includeTldrs": true,
    "performTitleMatch": true,
    "includeBadges": true,
    "tldrModelVersion": "v2.0.0",
    "getQuerySuggestions": false
}
'''
# post = json.loads(data)
# res = requests.post(url, json=post)
# res.text

In [5]:
import requests
import json
import multiprocessing as mp
import numpy as np
import pandas as pd

from time import sleep, time


def timer(fun):
  def warper(*args,**kwargs):
    start = time()
    d = fun(*args,**kwargs)
    end = time()
    print(f"[{fun.__name__}]>> Demorou {round(end-start,4)}s")
    return d
  return warper



class SearchWeb():
  def __init__(self, search="Machine Learning+Deep Learning", poolCPU = 4, sleeptry=5, save = False, **kwargs):
     
    self.sleeptry = sleeptry
    self.poolCPU = poolCPU
    self.badcall = []
    self._start = True

    self.saveFile = save
    self._search = search
    self._page = kwargs.get('page', 1)
    self._sort = kwargs.get('sort', "relevance")
    self._authors = kwargs.get('authors', [])
    self._coAuthors = kwargs.get('coAuthors', [])
    self._venues = kwargs.get('venues', ['PloS one', 'AAAI', 'Scientific reports', 'IEEE Access', 'ArXiv', 'Expert Syst. Appl.', 'ICML', 'Neurocomputing', 'Sensors', 'Remote. Sens.'])
    self._yearFilter = kwargs.get('yearFilter', None) # {"min": 2008,"max": 2021}
    self._requireViewablePdf = kwargs.get('requireViewablePdf', False)
    self._publicationTypes = kwargs.get('publicationTypes', ["ClinicalTrial", "CaseReport", "Editorial","Study","Book","News","Review","Conference","LettersAndComments","JournalArticle"])
    self._fieldsOfStudy = kwargs.get('fieldsOfStudy', ["biology","art","business","computer-science","chemistry","economics","engineering","environmental-science","geography","geology","history","materials-science","mathematics","medicine","philosophy","physics","political-science","psychology","sociology"])
    self._useFallbackRankerService = kwargs.get('useFallbackRankerService', False)
    self._useFallbackSearchCluster = kwargs.get('useFallbackSearchCluster', False)
    self._hydrateWithDdb = kwargs.get('hydrateWithDdb', True)
    self._includeTldrs = kwargs.get('includeTldrs', True)
    self._performTitleMatch = kwargs.get('performTitleMatch', True)
    self._includeBadges = kwargs.get('includeBadges', True)
    self._tldrModelVersion = kwargs.get('tldrModelVersion', 'v2.0.0')
    self._getQuerySuggestions = kwargs.get('getQuerySuggestions', False)


    self.post = {
    "page": self._page, 
    "pageSize": 10,
    "queryString": self._search,
    "sort": self._sort,
    "authors": self._authors,
    "coAuthors": self._coAuthors,
    "venues": self._venues,
    "yearFilter": self._yearFilter,
    "requireViewablePdf": self._requireViewablePdf,
    "publicationTypes": self._publicationTypes,
    "externalContentTypes": [],
    "fieldsOfStudy": self._fieldsOfStudy,
    "useFallbackRankerService": self._useFallbackRankerService,
    "useFallbackSearchCluster": self._useFallbackSearchCluster,
    "hydrateWithDdb": self._hydrateWithDdb,
    "includeTldrs": self._includeTldrs,
    "performTitleMatch": self._performTitleMatch,
    "includeBadges": self._includeBadges,
    "tldrModelVersion": "v2.0.0",
    "getQuerySuggestions": self._getQuerySuggestions,
    }

    print('.post >>')
    print(self.post)

  def _query(self, page):
    url = "https://www.semanticscholar.org/api/1/search"
    post = self.post.copy()
    post["page"] = page
    return [requests.post(url, json=post), page ]

  def _json(self, res):
    return json.loads(res.text).copy()
  
  def _paperExtract(self, data):
    p = {
        "authors": [author[0]['name'] for author in data.get('authors',[{'name':None},None])],
        "id": data.get('id',None),
        "socialLinks": data.get('socialLinks',None),
        "title": data.get('title',{'text':None})['text'],
        "paperAbstract": data.get('paperAbstract',{'text':None})['text'],
        "year": data.get('year',{'text':None})['text'],
        "venue": data.get('venue',{'text':None})['text'],
        "citationContexts":data.get('citationContexts',None),
        "citationStats": data.get('citationStats',None),
        "sources":data.get('sources',None),
        "externalContentStats":data.get('externalContentStats',None),
        "journal":data.get('journal',None),
        "presentationUrls":data.get('presentationUrls',None),
        "links": data.get('links',None),
        "primaryPaperLink": data.get('primaryPaperLink',None),
        "alternatePaperLinks": data.get('alternatePaperLinks',None),
        "entities": [author['name'] for author in data.get('entities',[{'name':None}])],
        "entityRelations": data.get('entityRelations',None),
        "blogs":data.get('blogs',None),
        "videos":data.get('videos',None),
        "githubReferences": data.get('githubReferences',None),
        "scorecardStats": data.get('scorecardStats',None),
        "fieldsOfStudy":data.get('fieldsOfStudy',None),
        "pubDate":data.get('pubDate',None),
        "pubUpdateDate":data.get('pubUpdateDate',None),
        "badges":data.get('badges',None),
        "tldr":data.get('tldr',None)
        }
    return p

    

    # c['querySuggestions']
    # c['totalPages']
    # c['totalResults']
    
  def _save(self):
    with open('./DATA.json', 'w') as fp:
      json.dump(self.all, fp)
    print("[Save] >> Save ate current directory, ./DATA.json")

  @timer
  def _extract(self, pool, data):
    try:
      self.papers_text = pool.map(self._json, data['Response'])
      self.all["Results"].extend([{"N_Papers":len(page['results']),
                                   "Page": page['query']['page'],
                                   "Papers": pool.map(self._paperExtract,
                                                      page['results'])} for page in self.papers_text])
      if self.saveFile:
        try:
          self._save()
        except:
          print("_save >> [Fail] to save.")

    except:
      print("_extract>> [Fail], see .papers_text to reextract content.")
      self.badcall.append(self.papers_text)
    
    if self._start:
      print(f"Total Results: {self.papers_text[0]['totalPages']}")
      print(f"Total Pages: {self.papers_text[0]['totalResults']}")
      print(f"Query Suggestions: {self.papers_text[0]['querySuggestions']}")
      print('\n')
      self._start = False

  def _data(self, data):
    if type(self.datasource) == str:
      self.datasource = data
    else:
      self.datasource = pd.concat([self.datasource, data])

  @timer
  def _runtime(self, pool, pages):
    while True:
      res = pool.map(self._query, pages)
      self.codes = [[x[0], x[1],x[0].status_code] for x in res]
      resultData = pd.DataFrame(self.codes, columns=["Response", "Page", "Code"])
      resultData.set_index("Page")
      
      if resultData.query("Code !=200").size == 0:
        self._data(resultData)
        self._extract(pool, resultData.query("Code ==200"))
        break
      else:
        print("Bad call of pages:")
        self._data(resultData.query("Code == 200"))
              # self.datasource.append(resultData.query("Code ==200"))
        pages = resultData.query("Code !=200").index.values.tolist()
        print(pages)
        print(f"Tentando de novo daqui a {self.sleeptry/60} min...")
        self._extract(pool, resultData.query("Code ==200"))
        sleep(self.sleeptry)

    # self._extract(pool, self.datasource)


  @timer
  def get(self, n = 10):
    self.post["pageSize"] = 10
    self.all = {"Results": []}
    self.mult = ''
    self.datasource = ''
    print("Searching...")
    print(self.all)

    with mp.Pool(self.poolCPU) as pool:
      if n > 10:
        pages = list(range(self._page, (n//10)+self._page))
      # for page in range(n//10):
        self._runtime(pool,pages)
          
        if n%10>0:
          pages = [n//10+self._page]
          self.post["pageSize"] = n%10

          self._runtime(pool, pages)
          
      else:
        # pass
        pages = [self._page]
        self.post["page"] = self._page
        self.post["pageSize"] = n

        self._runtime(pool, pages)

      # self._extract(pool, self.datasource)


# if __name__ == '__main__':
#   SearchWeb().get()

In [6]:
from_Webpage = SearchWeb(search= "Machine Learning+Deep Learning", sort= "total-citations", page=1, save=True)

.post >>
{'page': 1, 'pageSize': 10, 'queryString': 'Machine Learning+Deep Learning', 'sort': 'total-citations', 'authors': [], 'coAuthors': [], 'venues': ['PloS one', 'AAAI', 'Scientific reports', 'IEEE Access', 'ArXiv', 'Expert Syst. Appl.', 'ICML', 'Neurocomputing', 'Sensors', 'Remote. Sens.'], 'yearFilter': None, 'requireViewablePdf': False, 'publicationTypes': ['ClinicalTrial', 'CaseReport', 'Editorial', 'Study', 'Book', 'News', 'Review', 'Conference', 'LettersAndComments', 'JournalArticle'], 'externalContentTypes': [], 'fieldsOfStudy': ['biology', 'art', 'business', 'computer-science', 'chemistry', 'economics', 'engineering', 'environmental-science', 'geography', 'geology', 'history', 'materials-science', 'mathematics', 'medicine', 'philosophy', 'physics', 'political-science', 'psychology', 'sociology'], 'useFallbackRankerService': False, 'useFallbackSearchCluster': False, 'hydrateWithDdb': True, 'includeTldrs': True, 'performTitleMatch': True, 'includeBadges': True, 'tldrModelVe

In [7]:
# Retorna 2 papers com base nos parametros passados em SearchWeb() que constitui ().post
from_Webpage.get(n = 12)

Searching...
{'Results': []}
[Save] >> Save ate current directory, ./DATA.json
Total Results: 5102
Total Pages: 51022
Query Suggestions: []


[_extract]>> Demorou 0.0682s
[_runtime]>> Demorou 2.5923s
[Save] >> Save ate current directory, ./DATA.json
[_extract]>> Demorou 0.0175s
[_runtime]>> Demorou 3.7995s
[get]>> Demorou 6.4582s


In [8]:
# Tudo que vem com base em 1 paper (dict.)
from_Webpage.all["Results"][0]["Papers"][0]

{'alternatePaperLinks': [],
 'authors': ['Martín Abadi',
  'Ashish Agarwal',
  'P. Barham',
  'E. Brevdo',
  'Z. Chen',
  'C. Citro',
  'G. Corrado',
  'Andy Davis',
  'J. Dean',
  'M. Devin',
  'S. Ghemawat',
  'I. Goodfellow',
  'Andrew Harp',
  'Geoffrey Irving',
  'M. Isard',
  'Y. Jia',
  'R. Józefowicz',
  'Lukasz Kaiser',
  'M. Kudlur',
  'Josh Levenberg',
  'Dandelion Mané',
  'Rajat Monga',
  'Sherry Moore',
  'D. Murray',
  'Christopher Olah',
  'M. Schuster',
  'Jonathon Shlens',
  'Benoit Steiner',
  'Ilya Sutskever',
  'Kunal Talwar',
  'P. Tucker',
  'V. Vanhoucke',
  'Vijay Vasudevan',
  'F. Viégas',
  'Oriol Vinyals',
  'Pete Warden',
  'M. Wattenberg',
  'Martin Wicke',
  'Yuan Yu',
  'X. Zheng'],
 'badges': [{'id': 'OPEN_ACCESS'}],
 'blogs': [],
 'citationContexts': [],
 'citationStats': {'citationAcceleration': -0.5003223726627982,
  'citationVelocity': 1463.3333333333333,
  'citedByBuckets': [{'count': 2, 'endKey': 2014, 'startKey': 2014},
   {'count': 9, 'endKey': 

In [ ]:

#     self.saveFile = save
#     self._publicationTypes = kwargs.get('publicationTypes', ["ClinicalTrial", "CaseReport", "Editorial","Study","Book","News","Review","Conference","LettersAndComments","JournalArticle"])
#     self._fieldsOfStudy = kwargs.get('fieldsOfStudy', ["biology","art","business","computer-science","chemistry","economics","engineering","environmental-science","geography","geology","history","materials-science","mathematics","medicine","philosophy","physics","political-science","psychology","sociology"])
#     self._useFallbackRankerService = kwargs.get('useFallbackRankerService', False)
#     self._useFallbackSearchCluster = kwargs.get('useFallbackSearchCluster', False)
#     self._hydrateWithDdb = kwargs.get('hydrateWithDdb', True)
#     self._includeTldrs = kwargs.get('includeTldrs', True)
#     self._performTitleMatch = kwargs.get('performTitleMatch', True)
#     self._includeBadges = kwargs.get('includeBadges', True)
#     self._tldrModelVersion = kwargs.get('tldrModelVersion', 'v2.0.0')
#     self._getQuerySuggestions = kwargs.get('getQuerySuggestions', False)

# search = str(input('Pesquisar(ex. "Machine+Medicine"): '))
# page = int(input('Pagina de inicio a busca: '))
# sleeptry = int(input('Tempo de recall: '))
# print('Opções de filtro [Sort](ex. total-citations): "total-citations", "influence", "pub-date", "relevance"')
# sort = str(input('Sort: '))
# print("Opções de filtro [Venues](ex.[] ou ['.', '.']): ['PloS one', 'AAAI', 'Scientific reports', 'IEEE Access', 'ArXiv', 'Expert Syst. Appl.', 'ICML', 'Neurocomputing', 'Sensors', 'Remote. Sens.']")
# poolCPU = int(input('Quantos cores (CPU): '))
# years = input('Periodo de tempo (ex.{"min": 1999, "max": 2000}) : ')
# requireViewablePdf = input('Ter Pdf free (True or False):')
# requireViewablePdf = input('Ter Pdf free (True or False): ["ClinicalTrial", "CaseReport", "Editorial","Study","Book","News","Review","Conference","LettersAndComments","JournalArticle"]:')

# "False" == False

# # from_Webpage.papers_text[0]["results"][0]